In this notebook, we will cover downloading dmi harmonie forecasts and processing to a dfs2 file format used by MIKE SHE for hydrogeological modelling.
We will use the standard python libaries os, sys, time, glob, traceback, copy and datetime, along with third party libaries numpy, pandas, xarray, matplotlib, requests, pytz, pyproj, rasterio, and mikeio.

## Step 1: Install Conda

If you haven't already, install Anaconda or Miniconda. You can download them from [Anaconda](https://anaconda.org/) or [Miniconda](https://docs.anaconda.com/miniconda/).

## Step 2: Create a New Conda Environment

Open Anaconda prompt, then run the following command to create a new environment. We’ll name it hydro_forecast_env, but feel free to use any name you like:  
`conda create -n hydro_forecast_env python=3.11`

This command will create an environment with Python 3.11, which is compatible with all the libraries we’ll use.

## Step 3: Activate the Environment
`conda activate hydro_forecast_env`

## Step 4: Install Required Packages
Now that the environment is active, we’ll install the required libraries. Some libraries can be installed directly via Conda, while others require pip.  
`conda install numpy pandas xarray matplotlib requests pytz pyproj rasterio -c conda-forge`

mikeio is specific to MIKE SHE and not available through Conda, so we’ll use pip to install it:
`pip install mikeio`

## Step 5: Verify the Installation
To make sure everything is correctly installed, open a Python interpreter in the Conda environment and run the following code:

In [17]:
# Standard library imports
import os, sys, time, glob, traceback, copy
from datetime import datetime, timedelta, date

# Third-Party imports
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import requests
import pytz
from pyproj import CRS
from rasterio.enums import Resampling

# MIKE IO imports
import mikeio
from mikeio import ItemInfo, EUMType, EUMUnit

Before we can access harmonie forecast data we must create an account following this [guide](https://opendatadocs.dmi.govcloud.dk/en/Authentication) and request an API key for forecastedr. Then we define a string variable using our API key.

In [25]:
api_key = '<insert api key here>'

Next, we will dynamically identify the closest available forecast based on the current time and create a model run ID. This assumes each forecast becomes available approximately six hours after its scheduled time. Alternatively, when making the API call later, you can omit the model run ID to automatically download the latest available forecast. Using the same syntax, we could also iterate over filtered forecasts to download each forecast.

In [22]:
# Set up forecast hours and date for today and yesterday
date_today = datetime.today().date()
date_yesterday = date_today - timedelta(days=1)
forecast_hours = ['00', '06', '12', '18']


# Determine the cutoff time (six hours prior to now)
tback = datetime.now() - timedelta(hours=6)

# Generate forecast times for today and yesterday
forecast_datetimes = [
    datetime.combine(date, datetime.strptime(hour, '%H').time())
    for date in [date_yesterday, date_today]
    for hour in forecast_hours
]

# Filter forecast times that are fully uploaded
filtered_forecasts = [h for h in forecast_datetimes if h < tback]

# Find the closest forecast time to tback
closest_forecast = max(filtered_forecasts, default=tback)

# Create modelrun ID
modelrun = f'{date_today}T{closest_forecast.strftime("%H")}0000Z'

In [23]:
modelrun

'2024-11-08T000000Z'

Next, we’ll write the API call to download the model results. We’ll use a dictionary to organize the various components of the request. First, we specify the **API key**, followed by the **endpoint**, which directs to the EDR API for DMI. The **collection** points to the DINI HARMONIE model, and the **instance** references the specific model run using our dynamically generated model ID. The **bbox** targets a bounding box around Denmark, and **crs** specifies the coordinate reference system. We include relevant **parameters** needed for calculating potential evapotranspiration and precipitation. Finally, we set the **format** to NetCDF for easy data handling.

In [31]:
api_dict = {
"apikey": f"api-key={api_key}",
"endpoint": r"https://dmigw.govcloud.dk/v1/forecastedr",
"collection": r"/collections/harmonie_dini_sf/",
"instance": fr"instances/{modelrun}/", 
"bbox": "bbox?bbox=5.03,53.21,17.5,58.57",
"crs": "crs=crs84",
"parameter": [
    "temperature-2m",
    "wind-speed",
    "total-precipitation",
    "pressure-surface",
    "relative-humidity-2m",
    "net-short-wave-radiation-flux",
    "net-long-wave-radiation-flux",
    "global-radiation-flux"
],
"format": "f=NetCDF"
}

In this step, we’ll create a directory to store the downloaded NetCDF files. Using **os.makedirs**, we specify an **output folder** in the current working directory, ensuring it exists or is created if missing.

We then proceed to download the forecast data for each specified parameter. For each parameter in our **api_dict**, we construct the API request URL by combining the endpoint, collection, instance, bounding box, parameter name, coordinate reference system, file format, and API key.

For each parameter, we send a request to download its data in NetCDF format. If successful, we save the file to our output folder with a filename that includes the parameter name (e.g., 'harmonie_temperature-2m.nc'). If any request fails, an error message is printed, allowing us to troubleshoot connectivity issues with the EDR API.

In [36]:
# Create a folder for storing netcdfs
output_folder = fr'{os.getcwd()}\output'
os.makedirs(output_folder, exist_ok=True)

# Download parameter forecast to netcdfs
try:
    for par in api_dict["parameter"]:
        entry_url = f"{api_dict['endpoint']}{api_dict['collection']}{api_dict['instance']}{api_dict['bbox']}&parameter-name={par}&{api_dict['crs']}&{api_dict['format']}&{api_dict['apikey']}"
        response  = requests.get(entry_url)
        response.raise_for_status()
                
        netcdf_file = os.path.join(output_folder, f"harmonie_{par}.nc")
                
        with open(netcdf_file, "wb") as file:
            file.write(response.content)
        print( f"Saving harmonie_{par}.nc")
except requests.exceptions.RequestException as e:
    print(f"Failed to connect to EDR API: {e}")

Saving harmonie_temperature-2m.nc
Saving harmonie_wind-speed.nc
Saving harmonie_total-precipitation.nc
Saving harmonie_pressure-surface.nc
Saving harmonie_relative-humidity-2m.nc
Saving harmonie_net-short-wave-radiation-flux.nc
Saving harmonie_net-long-wave-radiation-flux.nc
Saving harmonie_global-radiation-flux.nc


## 1. Parameter Dictionary

In [ ]:
param = {
    'temp': [EUMType.Temperature, EUMUnit.degree_Celsius, 'Temperature', 'Instantaneous', 'temperature-2m'],
    'pres': [EUMType.Pressure, EUMUnit.kilopascal, 'Pressure', 'Instantaneous', 'pressure-surface'],
    'grad': [EUMType.Sun_radiation, EUMUnit.kJ_per_meter_pow_2_per_hour, 'Global radiation', 'MeanStepBackward', 'global-radiation-flux'], 
    'precip': [EUMType.Precipitation_Rate, EUMUnit.mm_per_hour, 'Precipitation rate', 'MeanStepBackward', 'total-precipitation'], 
    'ws': [EUMType.Wind_speed, EUMUnit.meter_per_sec, 'Wind speed', 'Instantaneous', 'wind-speed'],
    'rh': [EUMType.Relative_humidity, EUMUnit.percent, 'Relative humidity', 'Instantaneous', 'relative-humidity-2m'],
    'nlwrf': [EUMType.Sun_radiation, EUMUnit.kJ_per_meter_pow_2_per_hour, 'Net longwave radiation flux', 'MeanStepBackward', 'net-long-wave-radiation-flux'], 
    'nswrf': [EUMType.Sun_radiation, EUMUnit.kJ_per_meter_pow_2_per_hour, 'Net shortwave radiation flux', 'MeanStepBackward', 'net-short-wave-radiation-flux'],
    'makkink': [EUMType.Evapotranspiration_Rate, EUMUnit.mm_per_hour, 'Potential evapotranspiration', 'MeanStepBackward'],
    'penman_monteith': [EUMType.Evapotranspiration_Rate, EUMUnit.mm_per_hour, 'Potential evapotranspiration', 'MeanStepBackward'],
}

This dictionary maps each parameter (e.g., temperature, pressure, etc.) to a list containing information about:
- The mikeio type (e.g., Temperature, Pressure).
- The mikeio unit (e.g., degree Celsius, kilopascal).
- A mikeio item name (e.g., 'Temperature', 'Pressure').
- A mikeio step type (e.g., 'Instantaneous', 'MeanStepBackward').
- The initial parameter name used in the NetCDF files (e.g., 'temperature-2m').

These are not the initial units, but the units we will save the dfs2 files with. To see the initial units [check here.](https://opendatadocs.dmi.govcloud.dk/Data/Forecast_Data_Weather_Model_HARMONIE_DINI_IG)

## 2. Opening NetCDF Files as xarray Dataset

In [ ]:
netcdf_files = glob(f'{output_folder}/*.nc')
ds = xr.open_mfdataset(netcdf_files, combine='by_coords')

This uses the glob function to get a list of all .nc (NetCDF) files in the output_folder.  
The open_mfdataset function from xarray is used to load all the NetCDF files into a single dataset, combining them by coordinates (assuming they share the same grid).

## 3. Preparing Dataset

In [ ]:
# Renaming
rename_dict = {values[-1]: new_name for new_name, values in param.items() if len(values) > 4} 
ds = ds.rename(rename_dict)

# Assigning custom CRS
dmi_harmonie_crs = CRS.from_proj4("+proj=lcc +lat_1=55.5 +lat_2=55.5 +lat_0=55.5 "
                                  "+lon_0=-8 +x_0=0 +y_0=0 +a=6371229 +b=6371229 +units=m +no_defs")
ds.rio.write_crs(
    dmi_harmonie_crs.to_wkt(), 
    inplace=True
)

# Assigning spatial dimensions
ds.rio.set_spatial_dims(
    x_dim='projection_x_coordinate', 
    y_dim='projection_y_coordinate', 
    inplace=True
)
# Reproject dataset to UTM32N
ds = ds.rio.reproject(
    25832, 
    resolution=2000, 
    resampling=Resampling.bilinear
)
# Sptial subsetting
mask_x = np.logical_and(ds.x >= 440000, ds.x <= 900000)
mask_y = np.logical_and(ds.y >= 6035000, ds.y <= 6484487)
mask = np.logical_and(mask_x, mask_y)
ds = ds.where(mask, drop=True)

A `rename_dict` is created that maps the parameter name in the NetCDF files (e.g., 'temperature-2m') to a more readable name (e.g., 'temp') from the param dictionary. This rename operation is then applied to the dataset to ensure that the variables in the dataset have more user-friendly names. 

A custom Lambert Conformal Conic (LCC) projection using the PROJ.4 format is created and assigned to the dataset using the `rio.write_crs` function from rioxarray. The `to_wkt()` method converts the CRS to Well-Known Text (WKT) format, which is the required input for this function.  

The `set_spatial_dims` function is used to specify which dimensions represent the spatial coordinates (i.e., the X and Y axes). Here, it’s assumed that the dataset uses 'projection_x_coordinate' and 'projection_y_coordinate' for the horizontal axes.  
The dataset is reprojected to UTM Zone 32N (EPSG:25832) using the `rio.reproject` function. The resolution is set to 2000 meters, and the resampling method used for interpolation is bilinear, which is appropriate for continuous data like temperature or precipitation.  

Masking is done to limit the dataset to a specific geographical region. The bounds for the X and Y coordinates are set to cover the area of interest, which in this case Denmark. The `where` function is used to apply the mask, dropping data outside the specified region.  

## 4. Unit conversion and de-accumulating the accumulated parameters

In [ ]:
# Convert units
ds['pres'] = ds['pres'] * 1E-3   # Pa to kPa
ds['temp'] = ds['temp'] - 273.15 # Kelvin to Celsius 
        
# Create a list of accumulated parameters to de-accumulate
accumulated_vars = ['grad', 'nswrf', 'nlwrf', 'precip']
    
# De-accumulating the accumulated parameters
for var in accumulated_vars:
    data = ds[var].data                     # Extract original data
    ds[var].data[1:] = data[1:] - data[:-1] # Calculate difference with previous timestep
        
# Calculate net radiation and convert units for PET calculations
ds['rn']   = ds['nswrf'] + ds['nlwrf'] # Calculate net radiation
ds['grad'] = ds['grad'] * 1E-6         # Joul to Megajoul            
ds['rn']   = ds['rn']   * 1E-6         # Calculate Net radiation and to Megajoul

In preperation to calculating Penman-monteith and Makkink potential evapotranspiration, units are converted to match the FAO [guideline](https://www.fao.org/4/x0490e/x0490e05.htm#TopOfPage). The pressure (`pres`) variable is converted from Pascals (Pa) to kilopascals (kPa) by multiplying by 1 x 10-3 and temperature (`temp`) is coverted from Kelvin to Celsius by subtracting 273.15.   
For variables representing accumulated values (`grad`, `precip`, `nswrf`, `nlwrf`), the values are "de-accumulated" to show instantaneous changes per timestep, rather than cumulative totals. This is done by subtracting each timestep's value from the previous timestep. Spurious positive or negative values can sometimes appear in the de-accumulated values due to rounding in accumulation processes; this behavior is discussed in the ECMW [documentation](https://confluence.ecmwf.int/display/UDOC/Why+are+there+sometimes+small+negative+precipitation+accumulations+-+ecCodes+GRIB+FAQ).  
  
For the Penman-Monteith calculation, net radiation (`rn`) is computed as the sum of net shortwave (`nswrf`) and net longwave radiation fluxes (`nlwrf`). Finally, both `grad` and `rn` are converted from joules to megajoules by multiplying by 1 x 10−6 to match conventional units for evapotranspiration calculations.


## 5. Calculating Penman-monteith and Makkink PET

In [ ]:
def get_daylight_factors(timeseries, timezone = 'Europe/Copenhagen', lat = 55.686, lng = 12.570):
    """ 
    Calculates daylight factors (sunrise, sunset) for each timestep in the timeseries.
    """
    daylight_factors = []
    copenhagen_tz = pytz.timezone(timezone)
    for timestep in timeseries:
        ts_datetime = pd.to_datetime(timestep)
        
        # Get sunrise and sunset times
        response = requests.get('https://api.sunrise-sunset.org/json', 
                                params={'lat':lat, 'lng': lng, 'date': ts_datetime.date()})
        r = response.json()['results']
        
        daylight = 2 if copenhagen_tz.dst(ts_datetime, is_dst=None) else 1
        sunrise  = datetime.strptime(r['sunrise'], '%I:%M:%S %p').time()
        sunset   = datetime.strptime(r['sunset'], '%I:%M:%S %p').time()
        
        sunrise_hour = sunrise.hour + sunrise.minute / 60.0 + daylight
        sunset_hour  = sunset.hour + sunset.minute / 60.0 + daylight
        
        daylight_factors.append((sunrise_hour, sunset_hour))
        
    return daylight_factors

def calculate_penman_monteith(temp, rh, ws, rn, pres, timeseries):
    """
    Calculates Penman-Monteith potential evapotranspiration.
    """
    # Copy net radiation (rn) for calculating G flux
    G = copy.deepcopy(rn)
    daylight_factors = get_daylight_factors(timeseries)
    
    # Adjust G flux based on daylight hours
    for i, (sunrise, sunset) in enumerate(daylight_factors):
        hour = pd.to_datetime(timeseries[i]).hour + pd.to_datetime(timeseries[i]).minute / 60
        if sunrise <= hour < sunset:
            G[i, :, :] = 0.1 * rn[i, :, :]
        else:
            G[i, :, :] = 0.5 * rn[i, :, :]

    # Constants and derived values
    CP = 1.013e-3                                       # Specific heat of air [MJ kg-1 °C-1]
    λ  = 2.501 - 0.002361 * temp                        # Latent heat of vaporization [MJ kg-1]
    γ  = CP * pres / (0.622 * λ)                        # Psychrometric constant [kPa °C-1]
    es = 0.6108 * np.exp(17.27 * temp / (temp + 237.3)) # Saturation vapor pressure [kPa]
    ea = rh / 100 * es                                  # Actual vapor pressure [kPa]
    s  = (4098 * es) / (temp + 237.3) ** 2              # Slope of saturation vapor pressure curve [kPa °C-1]
    pet_factor = 37
    
    # Wind speed adjustment to 2 m height
    u2 = (4.87 / np.log(67.8 * 10 - 5.42)) * ws # Adjusted wind speed at 2 m height

    # Penman-Monteith equation (hourly)
    pm_pet = (0.408 * s * (rn - G) + γ * pet_factor / (temp + 273) * u2 * (es - ea)) / (s + γ * (1 + 0.34 * u2))

    # Clip negative values
    return pm_pet.clip(min=0)

def calculate_makkink(temp, rs, pres):
    """
    Calculates Makkink potential evapotranspiration.
    """
    # Constants and derived values
    CP = 1.013e-3                                       # Specific heat of air [MJ kg-1 °C-1]
    λ  = 2.501 - 0.002361 * temp                        # Latent heat of vaporization [MJ kg-1]
    γ  = (CP * pres) / (0.622 * λ)                      # Psychrometric constant [kPa °C-1]
    es = 0.6108 * np.exp(17.27 * temp / (temp + 237.3)) # Saturation vapor pressure [kPa]
    s  = (4098 * es) / (temp + 237.3) ** 2              # Slope of saturation vapor pressure curve [kPa °C-1]
    
    # Makkink equation
    makkink_pet = (0.65 * s * rs) / (λ * (s + γ))
    
    # Clip negative values 
    return makkink_pet.clip(min=0)

For more information about the equations, parameters and units, please check out FAO [documentation](https://www.fao.org/4/x0490e/x0490e05.htm#TopOfPage). These are setup for calculating PET on hourly basis, which is also the timestep interval for the HARMONIE forecast. Next, we simply call the functions to create a makkink and penman-monteith data variable. These will represent the hourly potential evapotranspiration in milimeters.

In [ ]:
ds['makkink'] = calculate_makkink(
    temp = ds['temp'], 
    rs   = ds['grad'], 
    pres = ds['pres'],
)

ds['penman_monteith'] = calculate_penman_monteith(
    temp       = ds['temp'], 
    rh         = ds['rh'], 
    ws         = ds['ws'], 
    rn         = ds['rn'], 
    pres       = ds['pres'], 
    timeseries = ds['time'].values, 
)

## 6. Saving as dfs2 files

In [ ]:
# Converting units to match mikeio available units
ds['grad']   = ds['grad']  * 1000         # Megajoul to Kilojoul
ds['rn']     = ds['rn']    * 1000         # Megajoul to Kilojoul
ds['nswrf']  = ds['nswrf'] * 1.00E-03     # Joul to Kilojoul
ds['nlwrf']  = ds['nlwrf'] * 1.00E-03     # Joul to Kilojoul
ds['precip'] = ds['precip'].clip(min = 0) # Clipping negative precip to 0

# Define mikeio geometry and timeseries
geometry  = mikeio.Grid2D(x=ds.x.data, y=ds.y.data[::-1], projection="NON-UTM")
timesteps = pd.DatetimeIndex(ds.time)
timestamp = datetime.strptime(modelrun, '%Y-%m-%dT%H%M%SZ')
timestamp = timestamp.strftime("%Y-%m-%dT%H") 

# Export climate variables as dfs2 files
for par in param:
    # Invert y-axis
    for i in range(0,len(ds.time)):
        ds[par].data[i,:,:] = np.flipud(ds[par].data[i,:,:])
    
    # Create mikeio dataarray   
    ds_mikeio = mikeio.DataArray(
        data = ds[par].data,  # Data
        time = timesteps,     # Time steps
        geometry = geometry,  # Grid geometry
        item = ItemInfo(
            param[par][2],    # EUMLable
            param[par][0],    # EUMType
            param[par][1],    # EUMUnit
            param[par][3],    # EUMTimeseries Type
        )
    )   
    ds_mikeio.to_dfs(f"{output_folder}/{par}_{timestamp}.dfs2")

This final code block prepares and exports climate data to .dfs2 format using mikeio. Each variable is processed individually, converted to appropriate units, and clipped if necessary, then saved in a format compatible with MIKE models. The variables `grad` and `rn` are converted from Megajoules to Kilojoules, `nswrf` and `nlwrf` are converted from Joules to Kilojoules and `precip` is clipped to remove negative values.  
The grid `geometry` uses the mikeio.Grid2D class to create a grid based on x and y coordinates, with the y-axis inverted to match MIKE’s format.  
The `timesteps` is a DatetimeIndex that matches the times in `ds`.  
For each parameter in param, the data is inverted along the y-axis (`np.flipud`) to align with the .dfs2 format requirements. A mikeio.DataArray is created to package the data along with time steps and grid geometry. The ItemInfo class is used to specify details like the unit label, type, and time series type for each variable, which is necessary for MIKE to interpret the file. Finally, the DataArray is saved to .dfs2 format with a filename based on the parameter name and timestamp.